In [ ]:
import pandas as pd
import numpy as np

In [31]:
# clean and write data to csv
cv = pd.read_csv('covid_data_raw.csv')
cv=cv[(cv["continent"]=="Europe") & (cv["indicator"] == "cases")]
cv.drop(["population", "country_code", "source", "rate_14_day", "cumulative_count", "indicator", "continent"], inplace=True, axis=1)
cv.to_csv("covid_data.csv", index=False)
cv.head()